Add 'tesla_price_data.csv' file from the same folder to this runtime.

In [1]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3
import talib as ta

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4227    0  4227    0     0   8715      0 --:--:-- --:--:-- --:--:--  8715
100  517k  100  517k    0     0   385k      0  0:00:01  0:00:01 --:--:--  609k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4235    0  4235    0     0   9625      0 --:--:-- --:--:-- --:--:--  9625
100  392k  100  392k    0     0   327k      0  0:00:01  0:00:01 --:--:-- 1435k


In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px


# Load data from CSV file
def data_loading(path):
  df = pd.read_csv(path, parse_dates=['Date'])
  return df


# Data Cleaning
def data_cleaning(df):
  print('is null?\n',df.isnull().sum())
  print('\n')
  print('is duplicate',df.duplicated().sum())
  df['Date'] = pd.to_datetime(df['Date'])


# Trends in the stock price
def get_trend(df):
  fig1 = go.Figure(data=go.Candlestick(x=df['Date'], open=df['Open'], high=df['High'], low=df['Low'], close=df['Close']))
  fig1.update_layout(title='TSLA', yaxis_title='Price',xaxis_rangeslider_visible=False)
  return fig1
  #fig1.show()


# Finding max and min price and Percentage increase
def min_max_price(df):

  min_price = df['Close'].min()
  min_date = df.loc[df['Close'] == min_price, 'Date'].values[0]


  max_price = df['Close'].max()
  max_date = df.loc[df['Close'] == max_price, 'Date'].values[0]


  # Find the current price and date
  current_price = df['Close'].iloc[-1]
  current_date = df['Date'].iloc[-1]

  # Print the results
  print("Minimum price was", min_price, "on", min_date)
  print("Current price is", current_price, "as of", current_date)
  print("Maximum price was", max_price, "on", max_date)

  #Percentage increase
  initial_investment = 100
  per_increase = ((current_price-min_price)/(min_price))*100
  current_value = initial_investment * (1 + per_increase / 100)
  # print(initial_investment,per_increase,current_value)

  print(f'If you invested {initial_investment}$ in this stock on {min_date} then the percentage increase is {per_increase}% and today it would be worth {current_value}')



# Calculating Yearly Growth
def annual_growth(df):
  df['Year'] = df['Date'].dt.year

  yearly_data = df.groupby('Year').agg({'Open': 'first', 'Close': 'last'})

  # Calculate the percentage change for each year
  yearly_data['Growth'] = (yearly_data['Close'] - yearly_data['Open']) / yearly_data['Open'] * 100

  colors = ['green' if growth > 0 else 'red' for growth in yearly_data['Growth']]

  fig = go.Figure(data=[go.Bar(x=yearly_data.index, y=yearly_data['Growth'],marker=dict(color=colors))])

  fig.update_layout(title='Annual Growth for TSLA', xaxis_title='Year', yaxis_title='Growth (%)')

  return fig
  # fig.show()



# Calculating Percentage change with different timeframes
def per_change(df):
  df['Pct_Change'] = df['Close'].pct_change()

  # Calculate rolling percentage change over different time periods
  df['5_Day_Perf'] = df['Pct_Change'].rolling(window=5).sum().shift(-4) * 100
  df['1_Month_Perf'] = df['Pct_Change'].rolling(window=21).sum().shift(-20) * 100
  df['3_Month_Perf'] = df['Pct_Change'].rolling(window=63).sum().shift(-62) * 100
  df['1_Year_Perf'] = df['Pct_Change'].rolling(window=252).sum().shift(-251) * 100
  df['3_Year_Perf'] = df['Pct_Change'].rolling(window=756).sum().shift(-755) * 100


  # Create horizontal bar chart of rolling percentage change
  fig = go.Figure()
  fig = go.Figure(go.Bar(
              x=[df['5_Day_Perf'].dropna().iloc[-1], df['1_Month_Perf'].dropna().iloc[-1], df['3_Month_Perf'].dropna().iloc[-1], df['1_Year_Perf'].dropna().iloc[-1],df['3_Year_Perf'].dropna().iloc[-1]],
              y=['5 Day Performance', '1 Month Performance', '3 Month Performance', '1 Year Performance', '3 Year Performance'],
              orientation='h',
              marker_color=['green' if perf > 0 else 'red' for perf in [df['5_Day_Perf'].dropna().iloc[-1], df['1_Month_Perf'].dropna().iloc[-1], df['3_Month_Perf'].dropna().iloc[-1], df['1_Year_Perf'].dropna().iloc[-1], df['3_Year_Perf'].dropna().iloc[-1]]]))
              
  fig.update_layout(title='Performance',
                    xaxis_title='Percentage Change',
                    yaxis_title='Time Period')
  return fig
  # fig.show()



# Find Volatility
def get_volatility(df):
  df['DailyChange'] = df['Close'] - df['Open']
  df['Volatility'] = df['DailyChange'].rolling(window=30).std()
  fig2 = go.Figure(data=go.Scatter(x=df['Date'], y=df['Volatility']))
  fig2.update_layout(title='TSLA Volatility', yaxis_title='Volatility')
  return fig2
  # fig2.show()



# Trading Volume analysis
def get_volume(df):
  volume_by_year = df.groupby(df['Date'].dt.year)['Volume'].sum().reset_index()
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=volume_by_year['Date'], y=volume_by_year['Volume'], mode='lines'))
  fig.update_layout(title='Yearly Trading Volume for TSLA', xaxis_title='Year', yaxis_title='Volume')
  return fig
  # fig.show()



# EMA Analysis
def get_EMA(df):
  df['EMA_50'] = ta.EMA(df['Close'],50)
  df['EMA_200'] = ta.EMA(df['Close'],200)
  fig4 = go.Figure()
  fig4.add_trace(go.Candlestick(x=df['Date'], open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='Price'))
  fig4.add_trace(go.Scatter(x=df['Date'], y=df['EMA_50'], name='50-day exponential moving average',line=dict(color='yellow')))
  fig4.add_trace(go.Scatter(x=df['Date'], y=df['EMA_200'], name='200-day exponential moving average',line=dict(color='black')))
  fig4.update_layout(title='TSLA Moving Averages', yaxis_title='Price',xaxis_rangeslider_visible=False)
  return fig4
  # fig4.show()



# RSI Analysis
def get_RSI(df):
  df['RSI'] = ta.RSI(df['Close'])
  df = df[df['Date'] >= '2020-01-01']

  fig6 = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1)
  fig6.add_trace(
      go.Scatter(x=df['Date'], y=df["Close"], name="Close"),
      row=1, col=1
  )

  fig6.add_trace(
      go.Scatter(x=df['Date'], y=df["RSI"], name="RSI",line=dict(color="violet")),
      row=2, col=1
  )

  fig6.update_layout(
      title="RSI TSLA",
  )

  fig6.update_layout(
      shapes=[
          dict(
              type="line",
              xref="paper",
              yref="y2",
              x0=0,
              y0=70,
              x1=1,
              y1=70,
              line=dict(color="red", width=1, dash="dash")
          ),
          dict(
              type="line",
              xref="paper",
              yref="y2",
              x0=0,
              y0=30,
              x1=1,
              y1=30,
              line=dict(color="green", width=1, dash="dash")
          )
      ]
  )

  return fig6
  # fig6.show()




# MACD Analysis
def get_MACD(df):
    df = df[df['Date'] >= '2020-01-01']

    macd, macdsignal, macdhist = ta.MACD(df['Close'])

    fig7 = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[0.6, 0.4])
    fig7.add_trace(go.Candlestick(x=df['Date'], open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='Candlestick'), row=1, col=1)
    fig7.add_trace(go.Scatter(x=df['Date'], y=macd, name='MACD',line=dict(color='blue')), row=2, col=1)
    fig7.add_trace(go.Scatter(x=df['Date'], y=macdsignal, name='Signal',line=dict(color='orange')), row=2, col=1)
    color = np.where(macdhist >= 0, 'green', 'red')
    fig7.add_trace(go.Bar(x=df['Date'], y=macdhist, name='Histogram', marker=dict(color=color)), row=2, col=1)
    fig7.update_layout(xaxis_rangeslider_visible=False)
    fig7.update_yaxes(title_text='MACD', row=2, col=1)

    return fig7
    # Show the figure
    # fig7.show()




Add 'tesla_price_data.csv' file to the runtime

In [3]:
# Add 'tesla_price_data.csv' file to the runtime
path = '/content/tesla_price_data.csv'
df = data_loading(path)
data_cleaning(df)
min_max_price(df)

# Article
article = """
<!DOCTYPE html>
<html>
<head>
	<title>TSLA Analysis</title>
<style>
		body {{
			margin: 0 100px;
		}}
	</style>
</head>
<body>
  <p><strong><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 26px;'><u>Tesla&apos;s Stock Market Performance: Insights from Technical Analysis</u></span></strong></p>
  <p><br></p>
  <p><strong><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px;'>About TSLA:</span></strong></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px;'>Tesla Inc. (TSLA) is an American electric vehicle and clean energy company, founded in 2003. Tesla has become one of the most valuable car manufacturers in the world. Its stock is traded on the NASDAQ stock exchange under the ticker symbol TSLA.</span></p>
  <p><br></p>
  <p><br></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(0, 0, 0);'>If you had invested&nbsp;</span><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 24px; color: rgb(40, 50, 78);'><strong>$100</strong></span><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(40, 50, 78);'>&nbsp;</span><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(0, 0, 0);'>in Tesla&apos;s stock four years ago, the investment would have a current value of&nbsp;</span><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 24px; color: rgb(65, 168, 95);'><strong>$1346</strong></span><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(0, 0, 0);'>, resulting in a percentage increase of&nbsp;</span><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 24px; color: rgb(65, 168, 95);'><u>1246%</u></span><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(0, 0, 0);'>.</span></p>
  <p><br></p>
  <p><br></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 22px; color: rgb(0, 0, 0);'><strong>Let us analyze TSLA to gain deeper insights and understanding:</strong></span></p>
  <div>{}</div>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>Starting with overall trend, we can see that the stock price has been quite volatile over the time period of 5 years. There are many large fluctuations in the price, which suggests that TSLA stock is a high-risk investment. However, we can also see that after 2022, the trend is downwards.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>An interesting insight we can derive from the chart is the presence of several mini-trends. These represent sudden, sharp increases/decrease in price that occur over a short period of time. These could by caused by a variety of factors, such as earnings reports or other news about the company.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(0, 0, 0);'>Key Takeaway:- Negative Trend after 2022</span></p>
  
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>One potential strategy for investors who are interested in TSLA stock could be to take advantage of these mini-trends. Investors could potentially make a significant profit by capturing these trends.</span></p>
  <p><br></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(0, 0, 0);'><em>But the question here is, how to identify such movements?</em></span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>For the average person, analyzing such vast amount of data and possessing financial knowledge necessary to engage in trading can be a daunting task. Their primary concern may lie in determining how much to invest and what the annual return on investment will be.</span></p>
  <p><br></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(0, 0, 0);'><em>Let us begin by examining the annual growth of the stock.</em></span></p>
  <p><br></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 22px; color: rgb(0, 0, 0);'><strong>Annual Growth:</strong></span></p>
  <div>{}</div>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>TSLA had a particularly massive strong growth year in 2020, with a growth percentage of over 730%. This was likely due to a combination of factors, including increased demand for electric vehicles and a growing interest in sustainable energy.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>However, in contrast, the year 2022 witnessed a poor growth percentage of -67% for TSLA. This decline in growth could potentially be explained by heightened competition within the electric vehicle market or other external factors.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>Overall, the annual growth chart is positive and it provides a useful way to track the performance of the TSLA stock over time and identify trends in its growth.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(0, 0, 0);'>Key Takeaway:- Positive annual growth</span></p>

  <p><br></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(0, 0, 0);'><em>Once we have established a positive trend in the overall growth of the stock, it may be worthwhile to examine its recent performances. Let us now turn our attention to analyzing some of the TSLA stock&apos;s recent performances.</em></span></p>
  <p><br></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 22px; color: rgb(0, 0, 0);'><strong>Performance:</strong></span></p>
  <div>{}</div>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>Based on the chart, it is evident that the recent performances of TSLA stock have been mixed. The 5-day performance has been positive, as indicated by the green bar. However, the 1-month, 3-month, and 1-year performances have been negative, as indicated by the red bars. On the other hand, the 3-year performance has been positive and has been the highest.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>From this, we can conclude that while the stock had short-term setbacks, its long-term growth trend has remained positive. This information can be particularly useful for investors who wish to monitor the stock&apos;s performance over various time periods. By comparing the rolling percentage change for each period, investors can gain insights into the recent and long-term performance of the stock. This information can help investors make informed decisions on whether to hold, sell, or purchase the stock.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(0, 0, 0);'>Key Takeaway:- Short-Term Performance: Negative, Long-Term Performance: Positive</span></p>

  <p><br></p>  
  <p><br></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(0, 0, 0);'><em>Investing in the stock market always carries a certain level of risk. However, taking calculated risks can lead to greater success.&nbsp;</em></span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>One way to measure risk is VOLATILITY.</span></p>
  <p><span style="font-family: 'Trebuchet MS', Helvetica, sans-serif;font-size: 18px; color: rgb(0, 0, 0)">In a 2020 report, Crestmont Research studied the historical relationship between stock market performance and volatility. For its analysis, Crestmont used the average range for each day to measure the volatility of the Standard &amp; Poor&apos;s 500 Index (S&amp;P 500). Their research found that higher volatility corresponds to a higher probability of a declining market, while lower volatility corresponds to a higher probability of a rising market.</span></p>
  <p><span style="font-family: 'Trebuchet MS', Helvetica, sans-serif;font-size: 18px; color: rgb(0, 0, 0)">Investors can use this data on long-term stock market volatility to align their portfolios with the associated expected returns.</span></p>
  <p><br></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 22px; color: rgb(0, 0, 0);'><strong>Volatility:</strong></span></p>
  <div>{}</div>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>The plot presented shows the volatility of TSLA stock over time, with volatility being defined as the degree of variation in a stock&apos;s price over time. A higher volatility implies that the stock price is fluctuating rapidly, which can lead to higher gains or losses.&nbsp;</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>The plot illustrates a significant increase in TSLA&apos;s volatility between 2021-2022, with a sudden spike indicating that the stock price has become more unpredictable and risky. This information is crucial for investors as they may need to adjust their investment strategies accordingly. It&apos;s worth noting that the volatility has stabilized somewhat since its peak in early 2022.&nbsp;</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>It&apos;s important to remember that past performance is not a guarantee of future results.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(0, 0, 0);'>Key Takeaway:- Stabilized volatility</span></p>

  <p><br></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 22px; color: rgb(0, 0, 0);'><strong>Trading Volume:</strong></span></p>
  <div>{}</div>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>The yearly trading volume plot of TSLA demonstrates an intriguing trend, which is of interest to investors and market analysts alike.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>The chart reveals that the trading volume has been consistently increasing since 2018, with a significant surge in 2020. The primary contributing factor to this increase is likely the heightened interest in electric vehicle companies and TSLA&apos;s success in the market.&nbsp;</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>It is also important to note the volatility of the trading volume, with a decline following the peak in 2020, followed by a range until 2022, and then a decrease. The trading volume is a key indicator of market sentiment, and the plot illustrates that TSLA&apos;s popularity has not been on a steady rise in recent years. This information can provide investors with insights into the market trends, which can be used to inform their investment decisions.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(0, 0, 0);'>Key Takeaway:- Trading Volume decreasing: Less popular</span></p>

  <p><br></p>
  <p><br></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(0, 0, 0);'><em>So to answer the same question, &quot; How to identify such movements/trends &quot;,&nbsp;</em><em>Let us get into some technical analysis.</em></span></p>
  <p><br></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 24px; color: rgb(0, 0, 0);'><strong>Technical Indicators:</strong></span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 22px; color: rgb(0, 0, 0);'><strong>EMA:</strong></span></p>
  <div>{}</div>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>To effectively analyze the market, it is essential to identify trends. This is where Exponential Moving Averages (EMA) come into play. The Moving Average chart depicts the TSLA stock price (candlestick chart) plotted against the 50-day EMA (in yellow) and the 200-day EMA (in black), which track the short-term and long-term trends, respectively. When the 50-day moving average crosses above the 200-day moving average, indicating a potential trend reversal and a bullish signal. Conversely, when the 50-day moving average crosses below the 200-day moving average, indicating a potential trend reversal and a bearish signal.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>Upon analyzing this chart, we can observe 50 EMA crossing below 200 EMA during October 2022, which indicates a bearish signal. The 200-day EMA acts as a resistance, as evidenced by the numerous attempts made by the market to break it. This suggests that bears currently have an upper hand in the market.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(0, 0, 0);'>Key Takeaway:- 200 EMA acting as Resistance</span></p>

  <p><br></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 22px; color: rgb(0, 0, 0);'><strong>RSI:</strong></span></p>
  <div>{}</div>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>The Relative Strength Index (RSI) is a popular momentum indicator used to measure the strength of a security&apos;s price action by comparing its upward and downward movements in closing price. The RSI is displayed as an oscillator, ranging from 0 to 100, with values above 70 indicating overbought conditions and values below 30 indicating oversold conditions.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>In the plot provided, the red dashed lines represent the overbought levels, while the green dashed lines represent the oversold levels for the RSI. It is observed that when the RSI value crosses above the 70 level significantly during November 2021, the price of TSLA tends to decline shortly after. Similarly, when the RSI value drops below the 30 level significantly during December 2022, the price of TSLA tends to rise shortly after.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>Traders can use the RSI to identify potential overbought and oversold conditions in the market and utilize that information to make more informed trading decisions.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(0, 0, 0);'>Key Takeaway:- RSI near oversold</span></p>
  
  <p><br></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(0, 0, 0);'><em>Till now, we know different ways to identify a trend but it is very important to know when to enter or exit. So for this, MACD is used.</em></span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 22px; color: rgb(0, 0, 0);'><strong>MACD:</strong></span></p>
  <div>{}</div>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>The <b>Moving Average Convergence/Divergence</b> is a trend-following momentum indicator that shows the relationship between two moving averages of a security&apos;s price.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>The MACD chart consists of two lines, the MACD line and the signal line, and a histogram that represents the difference between the two lines. A bullish signal is generated when the MACD line crosses above the signal line, indicating that the price may rise. Conversely, a bearish signal is generated when the MACD line crosses below the signal line, indicating that the price may fall.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>By using the MACD indicator, traders can identify potential buying and selling opportunities by looking for crossovers between the MACD and signal lines, as well as changes in the histogram bars, to make more informed trading decisions.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 18px; color: rgb(0, 0, 0);'>We can observe that MACD line crossed below signal line at a higher level during November 2021, which resulted into bearish trend. In the same way, during May 2022, MACD crossed above signal line, which resulted into a small bullish move.</span></p>
  <p><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 20px; color: rgb(0, 0, 0);'>Key Takeaway:- MACD near neutral line going downwards</span></p>

  <p><br></p>
  <p><br></p>
  <p><strong><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 22px;'>Key Takeaways:</span></strong></p>
  <ul>
      <li style='font-family: "Trebuchet MS", Helvetica, sans-serif;font-size: 18px'>Negative trend after 2022</li>
      <li style='font-family: "Trebuchet MS", Helvetica, sans-serif;font-size: 18px'>Positive annual growth</li>
      <li style='font-family: "Trebuchet MS", Helvetica, sans-serif;font-size: 18px'>Short-Term Performance: Negative</li>
      <li style='font-family: "Trebuchet MS", Helvetica, sans-serif;font-size: 18px'>Long-Term Performance: Positive</li>
      <li style='font-family: "Trebuchet MS", Helvetica, sans-serif;font-size: 18px'>Stabilized volatility</li>
      <li style='font-family: "Trebuchet MS", Helvetica, sans-serif;font-size: 18px'>Trading Volume decreasing: Less popular</li>
      <li style='font-family: "Trebuchet MS", Helvetica, sans-serif;font-size: 18px'>200 EMA acting as Resistance</li>
      <li style='font-family: "Trebuchet MS", Helvetica, sans-serif;font-size: 18px'>RSI near oversold</li>
      <li style='font-family: "Trebuchet MS", Helvetica, sans-serif;font-size: 18px'>MACD near neutral line going downwards
  </li>
  </ul>
  <p><br></p>
  <p><br></p>
  <p><strong><span style='font-family: "Trebuchet MS", Helvetica, sans-serif; font-size: 22px;'>Conclusion:</span></strong></p>
  <p><span style="font-family: 'Trebuchet MS', Helvetica, sans-serif; font-size: 18px;">The trend for the investment after 2022 has been negative with a positive annual growth. However, the short-term performance has been negative while the long-term performance has remained positive. The volatility of the investment has stabilized, but the trading volume has decreased, indicating lower popularity among investors. The 200 EMA is acting as a resistance level, and the RSI is near oversold. However, the MACD is near the neutral line and going downwards, which could suggest a bearish trend in the near future. Investors should closely monitor these technical indicators before making any investment decisions.</span></p>
</body>
</html>
"""

# Embed visualizations in article
plot_html1 = get_trend(df).to_html(full_html=False, include_plotlyjs='cdn')
plot_html2 = annual_growth(df).to_html(full_html=False, include_plotlyjs='cdn')
plot_html3 = per_change(df).to_html(full_html=False, include_plotlyjs='cdn')
plot_html4 = get_volatility(df).to_html(full_html=False, include_plotlyjs='cdn')
plot_html5 = get_volume(df).to_html(full_html=False, include_plotlyjs='cdn')
plot_html6 = get_EMA(df).to_html(full_html=False, include_plotlyjs='cdn')
plot_html7 = get_RSI(df).to_html(full_html=False, include_plotlyjs='cdn')
plot_html8 = get_MACD(df).to_html(full_html=False, include_plotlyjs='cdn')

article = article.format(plot_html1, plot_html2, plot_html3, plot_html4, plot_html5, plot_html6, plot_html7, plot_html8)

# Save article to file
with open('article.html', 'w') as f:
    f.write(article)


is null?
 Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


is duplicate 0
Minimum price was 11.931333 on 2019-06-03T00:00:00.000000000
Current price is 160.610001 as of 2023-05-03 00:00:00
Maximum price was 409.970001 on 2021-11-04T00:00:00.000000000
If you invested 100$ in this stock on 2019-06-03T00:00:00.000000000 then the percentage increase is 1246.119507350939% and today it would be worth 1346.119507350939


After running every cell, one html file will br generated named ' article.html ' which is the required file.